https://www.nature.com/articles/s41598-021-00053-8#MOESM1

In [1]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'letmein'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [3]:
neo4j_query("""
CALL apoc.warmup.run
""")

,pageSize,totalTime,transactionWasTerminated,nodesPerPage,nodesTotal,nodePages,nodesTime,relsPerPage,relsTotal,relPages,...,stringPropRecordsTotal,stringPropPages,stringPropsTime,arrayPropsPerPage,arrayPropRecordsTotal,arrayPropPages,arrayPropsTime,indexesLoaded,indexPages,indexTime
0,8192,8055,False,0,10945577,28618,1343,0,27064293,114223,...,0,0,0,0,0,0,0,False,0,0


In [4]:
neo4j_query("""
CALL apoc.meta.stats
YIELD nodeCount, relCount, labels, relTypesCount
""")

,nodeCount,relCount,labels,relTypesCount
0,10945577,27064293,"{'Category': 6, 'Transaction': 6071027, 'Trade...","{'SOLD': 17096794, 'IN_COLLECTION': 4337718, '..."


In [ ]:
# NFTs

In [18]:
neo4j_query("""
MATCH (t:Transaction)
RETURN t.Datetime_updated.year as year, 
       count(*) as transactions, 
       sum(t.Price_USD) as total_volume, 
       avg(t.Price_USD) as average_price
ORDER BY year 
""")

,year,transactions,total_volume,average_price
0,2017,253100,1.829279e+07,72.274933
1,2018,449373,1.667865e+07,37.118441
2,2019,746489,1.867730e+07,25.086969
3,2020,1245954,7.762350e+07,62.551672
4,2021,3376111,7.559576e+08,223.996997


In [11]:
neo4j_query("""
MATCH (t:Transaction)
WHERE exists(t.Price_USD)
RETURN CASE WHEN t.Price_USD > 1 THEN true ELSE false END AS moreThanDollar, count(*) AS count
""")

,moreThanDollar,count
0,False,2730965
1,True,3331779


In [10]:
neo4j_query("""
MATCH (t:Transaction)
WHERE exists(t.Price_USD) AND t.Price_USD > 1
RETURN apoc.agg.statistics(t.Price_USD)
""")

,apoc.agg.statistics(t.Price_USD)
0,"{'total': 3331779, 'min': 1.0000050750633935, ..."


In [7]:
neo4j_query("""
MATCH (n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists(t.Price_USD)
WITH n, t.Price_USD as price
ORDER BY price DESC LIMIT 5
RETURN n.ID_token as token_id, n.Image_url_1 as image_url, price
""")

,token_id,image_url,price
0,7804,https://lh3.googleusercontent.com/UTfTs8VI5u3t...,7.501893e+06
1,3100,https://lh3.googleusercontent.com/PSZKMjmI6NBD...,7.501893e+06
2,0,https://lh3.googleusercontent.com/MWyOpZRNd0Ln...,2.684347e+06
3,3011,https://lh3.googleusercontent.com/tblRChM9Ych3...,1.723131e+06
4,14,https://lh3.googleusercontent.com/p-yKO6UHEXnC...,1.643615e+06


In [13]:
neo4j_query("""
MATCH (collection)<-[:IN_COLLECTION]-(n:NFT)<-[:FOR_NFT]-(t:Transaction)
WHERE exists (t.Price_USD)
RETURN collection.Collection AS collection, 
       avg(t.Price_USD) AS averagePrice, 
       count(distinct n) AS numberOfNfts
ORDER BY averagePrice DESC LIMIT 5""")

,collection,averagePrice,numberOfNfts
0,Saturdaynightlive,360856.408928,1
1,Trippderrickbarnesxflipkick,118838.578811,1
2,Eulerbeat,54005.956071,73
3,Ladyjdayxflipkick,41976.350888,1
4,Narcissus,30652.488391,6


In [15]:
neo4j_query("""
MATCH (c:Collection)<-[:IN_COLLECTION]-(n)<-[:FOR_NFT]-(t:Transaction)
WHERE c.Collection = "Cryptoapes"
RETURN n.Name AS nft, avg(t.Price_USD) AS averagePrice, n.Image_url_1 as ImageUrl
ORDER BY averagePrice DESC
LIMIT 5
""")

,nft,averagePrice,ImageUrl
0,CryptoApes #89 #LEGEND #Bitcoin Edition,7349.034825,https://lh3.googleusercontent.com/9ga6gDR2QGO8...
1,CryptoApes #21,942.397500,https://lh3.googleusercontent.com/Uf2h7-sJ6MEK...
2,CryptoApes #185 #Suicide Squad Edition,821.747500,https://lh3.googleusercontent.com/0oYhplrhqFul...
3,CryptoApes #182 #RARE,792.667500,https://lh3.googleusercontent.com/D-CVcM9bmfBc...
4,CryptoApes #107 #RARE,594.353182,https://lh3.googleusercontent.com/NNxiQM_e2hSR...


In [21]:
neo4j_query("""
match (t:Trader)
return coalesce(t.username, t.address) as username,
       size((t)-[:BOUGHT]->()) as bought,
       size((t)-[:SOLD]->()) as sold
ORDER BY bought + sold desc limit 5;
""")

,username,bought,sold
0,0x76481caa104b5f6bccb540dae4cefaf1c398ebea,130231,192586
1,0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa,0,149142
2,0x4FabDA,28,49338
3,0xfc624f8f58db41bdb95aedee1de3c1cf047105f1,1976,43571
4,StrongHands,43384,437


https://opensea.io/StrongHands

In [22]:
neo4j_query("""
match (t:Trader)
OPTIONAL MATCH (t)-[:BOUGHT]->(bt)
WITH t, sum(bt.Price_USD) AS boughtVolume
OPTIONAL MATCH (t)-[:SOLD]->(st)
WITH t, boughtVolume, sum(st.Price_USD) AS soldVolume
RETURN t.username as username, t.address as address,boughtVolume, soldVolume
ORDER BY boughtVolume + soldVolume
DESC LIMIT 5
""")

,username,address,boughtVolume,soldVolume
0,NullAddress,0x0000000000000000000000000000000000000000,5.506430e+07,3.142260e+03
1,None,0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa,0.000000e+00,2.660329e+07
2,SethS,0x6611fe71c233e4e7510b2795c242c9a57790b376,8.304005e+05,1.314407e+07
3,Pranksy,0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459,3.931823e+06,9.985689e+06
4,SomniumSpace,0xb98cdacd006b9d47c37ca63cc86f916ee23fc550,1.166480e+03,1.310994e+07


https://ethereum.stackexchange.com/questions/98354/why-are-all-nfts-created-from-null-address-on-opensea
    
/// @dev This emits when ownership of any NFT changes by any mechanism.
///  This event emits when NFTs are created (`from` == 0) and destroyed
///  (`to` == 0). Exception: during contract creation, any number of NFTs
///  may be created and assigned without emitting Transfer. At the time of
///  any transfer, the approved address for that NFT (if any) is reset to none.
event Transfer(address indexed _from, address indexed _to, uint256 indexed _tokenId);  

https://opensea.io/NullAddress

In [25]:
neo4j_query("""
MATCH (t:Trader)-[:SOLD]->(st:Transaction)-[:FOR_NFT]->(nft)
WHERE st.Price_USD > 100000
MATCH (t)-[:BOUGHT]->(bt:Transaction)-[:FOR_NFT]->(nft)
WHERE st.Datetime_updated_seconds > bt.Datetime_updated_seconds
RETURN t.username as trader, nft.Image_url_1 as nft, nft.ID_token AS tokenID, 
       st.Datetime_updated_seconds AS soldTime, 
       st.Price_USD AS soldAmount, 
       bt.Datetime_updated_seconds as boughtTime, 
       bt.Price_USD AS boughtAmount,
       st.Price_USD - bt.Price_USD AS difference
ORDER BY difference DESC LIMIT 5""")

,trader,nft,tokenID,soldTime,soldAmount,boughtTime,boughtAmount,difference
0,None,https://lh3.googleusercontent.com/tblRChM9Ych3...,3011,2021-04-27T15:44:50.000000000+00:00,1723131.135,2021-03-12T01:00:13.000000000+00:00,712808.00,1010323.135
1,PixelVault_,https://lh3.googleusercontent.com/L_N2IeXJDM7D...,12,2021-03-31T15:39:49.000000000+00:00,883388.375,2021-03-18T19:09:05.000000000+00:00,99253.00,784135.375
2,None,https://lh3.googleusercontent.com/l6EtSH2XoyOi...,2306,2021-04-18T11:04:44.000000000+00:00,994856.440,2021-02-11T04:01:32.000000000+00:00,220262.50,774593.940
3,TokenAngels,https://lh3.googleusercontent.com/tblRChM9Ych3...,3011,2021-03-12T01:00:13.000000000+00:00,712808.000,2020-10-12T18:11:59.000000000+00:00,15981.84,696826.160
4,Pranksy,https://lh3.googleusercontent.com/1Tc5JdWMknIk...,2140,2021-03-02T01:02:54.000000000+00:00,1147815.000,2021-02-22T10:44:16.000000000+00:00,676912.00,470903.000


https://www.quora.com/How-can-there-be-multiple-NFTs-for-a-single-piece-of-digital-art-having-a-hard-time-grasping-the-concept-of-how-NFTs-can-be-1-art-n-NFT

Two NFTs with the same image

# Network analysis


In [ ]:
neo4j_query("""
CALL gds.graph.create.cypher("nft", 
"MATCH (t:Trader) RETURN id(t) as id",
"MATCH (t1:Trader)-[:BOUGHT]->(t)<-[:SOLD]-(t2:Trader)
 RETURN id(t1) AS source, id(t2) as target, coalesce(sum(t.Price_USD),1) as weight")
""")

In [29]:
neo4j_query("""
CALL gds.wcc.stats("nft")
YIELD componentCount, componentDistribution
""")

,componentCount,componentDistribution
0,4814,"{'p99': 10, 'min': 1, 'max': 260279, 'mean': 1..."


In [32]:
neo4j_query("""
MATCH (t:Trader)-[:BOUGHT]->(tr)<-[:SOLD]-(t)
RETURN count(*) AS count, avg(tr.Price_USD) as averagePrice, max(tr.Price_USD) as maxPrice
""")

,count,averagePrice,maxPrice
0,2846,51.704155,3981.325


In [35]:
neo4j_query("""
MATCH p=(t:Trader)-[:BOUGHT]->()<-[:SOLD]-(t)
WHERE t.username = "grake"
RETURN p LIMIT 10
""")

,p
0,"((), ())"
1,"((), ())"
2,"((), ())"
3,"((), ())"


0xbad858a0cf09f210fcf35cbf83569178879b47f2

https://etherscan.io/address/0xbad858a0cf09f210fcf35cbf83569178879b47f2

In [38]:
neo4j_query("""
CALL gds.pageRank.stream("nft", {relationshipWeightProperty:'weight'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).username as username,gds.util.asNode(nodeId).address as address, score
ORDER BY score DESC
LIMIT 10""")

,username,address,score
0,None,0xa21037849678af57f9865c6b9887f4e339f6377a,14133.704590
1,None,0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa,2180.414617
2,None,0xa1e12defa6dbc8e900a6596083322946c03f01e3,2160.586811
3,YellowHeartFactories,0x95a08c297ad5861734c4440fd5ac6b80a8bf9228,1792.848498
4,GoldenBoy,0x3a92b1c299f1bdc57c94efe5d41b646a4bee13ef,1518.359196
5,None,0xba52c75764d6f594735dc735be7f1830cdf58ddf,1016.567104
6,None,yqzam.wam,830.472588
7,Carlini8,0x0d41f957181e584db82d2e316837b2de1738c477,767.799449
8,Pranksy,0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459,677.078612
9,ENS,0x0904dac3347ea47d208f3fd67402d039a3b99859,638.864374


https://etherscan.io/address/0xa21037849678af57f9865c6b9887f4e339f6377a

In [46]:
neo4j_query("""
MATCH (t:Trader)
WHERE t.address in ["0xa21037849678af57f9865c6b9887f4e339f6377a", "0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa", "0xa1e12defa6dbc8e900a6596083322946c03f01e3", "0x95a08c297ad5861734c4440fd5ac6b80a8bf9228"]
WITH t, coalesce(apoc.coll.sum([(t)-[:BOUGHT]->(b) | b.Price_USD]), 0.0) as boughtAmount,
        apoc.coll.sum([(t)-[:SOLD]->(s) | s.Price_USD]) as soldAmount
RETURN t.address AS address, boughtAmount, soldAmount, soldAmount - boughtAmount AS profit
""")

,address,boughtAmount,soldAmount,profit
0,0xa21037849678af57f9865c6b9887f4e339f6377a,53431.504311,3.480973e+06,3.427542e+06
1,0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa,0.000000,2.660329e+07,2.660329e+07
2,0xa1e12defa6dbc8e900a6596083322946c03f01e3,0.000000,2.030376e+06,2.030376e+06
3,0x95a08c297ad5861734c4440fd5ac6b80a8bf9228,5176.748500,7.096732e+05,7.044965e+05


https://opensea.io/0x327305a797d92a39cee1a225d7e2a1cc42b1a8fa